// voronoi  relaxation

//grid based hatching

// units are in mm

In [1]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line as draw_line
from skimage import data
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.transform import probabilistic_hough_line
import random 

# Load image"
# image_path = "/Users/nshelton/Pictures/gundam_title.png"
image_path = "/Users/nshelton/Pictures/gundam4x.jpeg"


image = cv2.imread(image_path)
print(image.shape)
# image[:,:,1] = image[:,:,0]
# image[:,:,2] = image[:,:,0]

image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_gray = cv2.resize(image_gray, (-1,-1), fx = 2, fy = 2)
 
# filtered = image_gray
sigma = 4
min_canny = 4
max_canny = 50
filtered = canny(image_gray, sigma, min_canny, max_canny).astype(np.uint8) * 255

cv2.imwrite("filtered.png", filtered)





(3192, 2256, 3)


True

In [2]:

# lines = probabilistic_hough_line(filtered, threshold=10, line_length=10, line_gap=2)

# canny_lines_img = np.zeros(image.shape, np.uint8)
# for line in lines:

#     cv2.line(canny_lines_img, line[0], line[1], tuple(color), 1, cv2.LINE_AA)


# cv2.imwrite("canny_lines.png", canny_lines_img)


# print(lines)

all_strokes_raster = np.ones(image_gray.shape) * 255

all_paths = []

while len(all_paths) < 5000 and filtered.sum() > 1000:
    print(filtered.sum(), "\t", len(all_paths),  end="\t\t\t\r")
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(filtered)

    def placeStroke(input_raster, start):
        img = input_raster.copy()
        path = []
        direction = np.pi
        pos = np.array(start)

        window = 1 
        max_val =input_raster[start[1], start[0]]

        while(max_val > 0):
            min_val, max_val, min_loc, max_loc  = cv2.minMaxLoc(img[pos[1] - window:pos[1] + window +1, pos[0] - window:pos[0] + window +1])
            pos += np.array(max_loc) - window 
            path.append(pos.copy())
            cv2.circle(img, pos, window-1, 0, -1)

        return path
    
    line_thickness = 2
    def rasterStroke(dims, path) :
        img = np.zeros(dims)
        for i in range(len(path)-1):
            a = (round(path[i][0]) , round(path[i][1]) )
            b = (round(path[i+1][0]) , round(path[i+1][1]) )
            cv2.line(img, a, b, 1, line_thickness, cv2.LINE_AA)
            
        return img

    
    new_stroke = placeStroke(filtered, max_loc)
    all_paths.append(new_stroke)
    stroke_raster = rasterStroke(image_gray.shape, new_stroke)

    filtered = filtered - stroke_raster * 255
    filtered = np.clip(filtered, 0, 255)
    all_strokes_raster = np.clip(all_strokes_raster.astype(np.int32) - stroke_raster.astype(np.int32) * 255, 0, 255).astype(np.uint8)

    # cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))
    # cv2.imwrite("filtered.png", filtered)
cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))




True

In [10]:
import cv2
import numpy as np

upscale = 10

def plotPaths(paths, path_scale=1):
    img = np.ones((297 * upscale, 420 * upscale, 3), np.uint8) * 255
    img = np.zeros((297 * upscale, 420 * upscale, 3), np.uint8) 

    for path in paths:

        color = (128 * int(random.random() * 2), 128 * int(random.random() * 2), 128 * int(random.random() * 2))
        # color = (255,255,255)
        for i in range(len(path) - 1):
            a = (
                round(path[i][0] * upscale * path_scale),
                round(path[i][1] * upscale * path_scale)
            )
            
            b = (
                round(path[i + 1][0] * upscale * path_scale),
                round(path[i + 1][1] * upscale * path_scale)
            )

            cv2.line(img, a, b, color, 1, cv2.LINE_AA)
        
        for i in range(len(path)):
            a = (
                round(path[i][0] * upscale * path_scale),
                round(path[i][1] * upscale * path_scale),
            )
            img[a[1], a[0]] = [0,255,0]

        if False:
            for p in path:
                a = (round(p[0] * upscale * path_scale), round(p[1] * upscale * path_scale))
                if a[0] < img.shape[1] and a[1] < img.shape[0]:
                    img[a[1], a[0], 0] = 255

    return img


# connect_close_paths(path_list)
path_list = [[p.tolist() for p in path] for path in all_paths]
render_img = plotPaths(path_list, 0.03)
cv2.imwrite("render.png", render_img)



True

In [4]:
from tqdm import tqdm


import numpy as np
from scipy.spatial import KDTree


def merge_one_path_kd(paths, threshold = 5):
    points = []
    for i in range(len(paths)):
        points.append(paths[i][0])
        points.append(paths[i][-1])

    tree = KDTree(points)
    

    for i in range(len(paths)):
        end_point = paths[i][-1]

        distance, idx = tree.query(end_point, k=2)
        other_idx = idx[1] // 2
        other_is_front = idx[1] %2 == 0
        if distance[1] < threshold and other_idx != i:

            other_path = paths[other_idx]
            if other_is_front: 
                paths[i] = paths[i] + other_path
                paths.pop(other_idx)
                break
            else:
                paths[i] = paths[i] + other_path[::-1]
                paths.pop(other_idx)
                break

        start_point = paths[i][0]
            
        distance, idx = tree.query(start_point, k=2)
        other_idx = idx[1] // 2
        other_is_front = idx[1] %2 == 0
        if distance[1] < threshold and other_idx != i:

            other_path = paths[other_idx]
            if other_is_front: 
                paths[i] = other_path[::-1] + paths[i]
                paths.pop(other_idx)
                break
            else:
                paths[i] = other_path + paths[i]
                paths.pop(other_idx)
                break

    return paths

def merge_one_path(paths, threshold = 5):

    for i in range(len(paths)):
        for j in range(len(paths)):
            if i == j: continue

            start1, end1 = paths[i][0], paths[i][-1]
            start2, end2 = paths[j][0], paths[j][-1]

            if (np.linalg.norm(np.array(end1) - np.array(start2)) < threshold ):
                new_paths = [paths[k] for k in range(len(paths)) if k!=i and k!=j]
                new_paths.append(paths[i]+paths[j])
                return new_paths

            if (np.linalg.norm(np.array(start1) - np.array(start2)) < threshold ):
                new_paths = [paths[k] for k in range(len(paths)) if k!=i and k!=j]
                new_paths.append(paths[i][::-1]+paths[j])
                return new_paths

            if (np.linalg.norm(np.array(end1) - np.array(end2)) < threshold ):
                new_paths = [paths[k] for k in range(len(paths)) if k!=i and k!=j]
                new_paths.append(paths[i]+paths[j][::-1])
                return new_paths

            if (np.linalg.norm(np.array(start1) - np.array(end2)) < threshold ):
                new_paths = [paths[k] for k in range(len(paths)) if k!=i and k!=j]
                new_paths.append(paths[j] + paths[i])
                return new_paths
            

    return paths

def close_loops(paths, threshold = 10):

    for i in range(len(paths)):
            start1, end1 = paths[i][0], paths[i][-1]
            if (np.linalg.norm(np.array(start1) - np.array(end1)) < threshold ):
                paths[i].append(start1)
    return paths

def smooth_closed_path(points, window_size=10):
    points_array = np.array(points)
    num_points = len(points)
    smoothed_points = []

    for i in range(num_points):
        # Calculate the indices for the moving window, considering wrap-around
        indices = [(i + j - window_size // 2) % num_points for j in range(window_size)]
        window_points = points_array[indices]
        avg_point = np.mean(window_points, axis=0)
        smoothed_points.append(avg_point)

    return smoothed_points

def smooth_path(points, window_size=2):
    points_array = np.array(points)
    smoothed_points = []

    for i in range(len(points)):
        start = max(0, i - window_size // 2)
        end = min(len(points), i + window_size // 2 + 1)
        window_points = points_array[start:end]
        avg_point = np.mean(window_points, axis=0)
        smoothed_points.append(avg_point)

    return smoothed_points

def smooth_all_2(paths, amount = 0.5) :
    smoothed_paths = []

    for k in range(len(paths)):
        smoothed = [paths[k][0]]
        for i in range(1, len(paths[k])-1):
            target = (paths[k][i-1]+paths[k][i+1])/2
            smoothed.append(paths[k][i] * (1 - amount) + target * amount)
        smoothed.append(paths[k][-1])

        path_is_closed =  np.array_equal(paths[k][0], paths[k][-1])
        
        if (path_is_closed):
            target = (paths[k][1]+paths[k][-2])/2
            smoothed[0] = smoothed[0] * (1 - amount) + target * amount
            smoothed[-1] = smoothed[0] 
        smoothed_paths.append(np.array(smoothed))
    
    return smoothed_paths

def smooth_all(paths, amount = 0.5) :
    result = []
    for path in paths:
        path = np.array(path)
 
        if np.all(path[0] == path[-1]) : 
            result.append(smooth_closed_path(path))
        else:
            result.append(smooth_path(path))
    return result

def simplify_all(paths, threshold=0):
    for i in range(len(path_list)):
        paths[i] = rdp(paths[i], threshold)
    return (paths)

def rdp(points, epsilon):
    """
    Simplifies a path using the Ramer-Douglas-Peucker algorithm.

    :param points: List of 2D points.
    :param epsilon: Distance threshold for simplification.
    :return: Simplified list of 2D points.
    """
    points = np.array(points)

    # Find the point with the maximum distance from the line formed by the first and last points
    dmax = 0
    index = 0
    end = len(points)
    for i in range(1, end - 1):
        d = perpendicular_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d

    # If the maximum distance is greater than epsilon, recursively simplify
    if dmax > epsilon:
        # Recursive call
        results1 = rdp(points[:index+1], epsilon)
        results2 = rdp(points[index:], epsilon)

        # Build the result list
        result = np.vstack((results1[:-1], results2))
    else:
        result = np.array([points[0], points[-1]])

    return result

def perpendicular_distance(point, line_start, line_end):
    """
    Calculate the perpendicular distance from a point to a line.

    :param point: The point (x, y).
    :param line_start: The start of the line (x, y).
    :param line_end: The end of the line (x, y).
    :return: The perpendicular distance.
    """
    if np.array_equal(line_start, line_end):
        return np.linalg.norm(point - line_start)
    else:
        return np.linalg.norm(np.cross(line_end - line_start, line_start - point)) / np.linalg.norm(line_end - line_start)

path_list = [[p.tolist() for p in path] for path in all_paths]

print(len(path_list))


for i in tqdm(range(len(path_list))):
    len_before = len(path_list)
    path_list = merge_one_path_kd(path_list)
    if len_before == len(path_list):
        break

print("merged to", len(path_list))



5000


 80%|████████  | 4009/5000 [00:47<00:11, 84.22it/s] 

merged to 991


In [5]:

path_list = close_loops(path_list)

print("closed loops", len(path_list))

path_list  = [np.array(path) for path in path_list]

for _ in range(1):
    path_list = smooth_all_2(path_list, 0.2)

print("smoothed 1", len(path_list))

path_list = simplify_all(path_list, 0.001)

print("simplify_all 1", len(path_list))

for _ in range(1):
    path_list = smooth_all_2(path_list, 0.5)

print("smoothed 2", len(path_list))

path_list = simplify_all(path_list, 0.1)
print("simplify 2", len(path_list))




closed loops 991
smoothed 1 991
simplify_all 1 991
smoothed 2 991
simplify 2 991


In [6]:
render_img = plotPaths(path_list, 0.05)
cv2.imwrite("render.png", render_img)



True

In [8]:
 
import json
import uuid

projects_folder = "/Users/nshelton/Hephaestus/projects"

def writePlot(paths):
    dictionary = {
        "created_time": "4/7/2024 4:42:27 PM",
        "modified_time": "4/7/2024 6:27:36 PM",
        "camera_position": [100, 100],
        "zoom": 10.702,
        "aspect": 0.563,
        "dom_element": {},
        "plot_models": [],
    }

    dictionary["plot_models"].append(
        {
            "position": {"x": 0, "y": 0},
            "paths": [path.tolist() for path in path_list],
            "scale": 0.03,
            "id": str(uuid.uuid4()),
            "state": "none",
        }
    )


    with open(projects_folder + "/gundam.json", "w") as outfile:
        
        json.dump(dictionary, outfile)


writePlot(path_list)

